In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_auc_score, roc_curve, precision_recall_curve
)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC, Precision, Recall
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stock_data = pd.read_csv("/content/drive/MyDrive/final_data.csv")
stock_data  = stock_data [stock_data ['date'].between('2004-01-01', '2024-12-31')]

In [ ]:
id_col = ["PERMNO", 'CUSIP', 'Ticker', 'SICCD','NAICS']
time_col = ['MthCalDt']
factor_col = ['mom_12','mom_6', 'vol_12', 'vol_6', 'rev_1', 'rvol_1', 'beta',"rsi_6", 'trend_strength']
market_col = [ 'qmj_safety','seas_11_15na',
              'ret_3_1', 'iskew_ff3_21d', 'rskew_21d', 'sti_gr1a','earnings_variability', 'nfna_gr1a',
              'seas_16_20an', 'corr_1260d']
fin_col = ['capxy', 'chechy', 'cshfdy', 'cshpry', 'dltry', 'dpcy',
       'epspxy', 'oibdpy', 'txty']

In [ ]:
train_set = stock_data[stock_data["year"] <= 2021]
vol_set = train_set[train_set["year"]>2018]
train_set = train_set[train_set["year"]<=2018]
test_set = stock_data[stock_data["year"]>2021]

In [ ]:
ml_col = factor_col+fin_col+market_col+["quarter", "naics", "ind_1"]

In [ ]:
X_train = train_set[ml_col].values
y_train = train_set["pred_cat"]
X_vol = vol_set[ml_col].values
y_vol = vol_set["pred_cat"]
X_test = test_set[ml_col].values
y_test = test_set["pred_cat"]

In [ ]:
def create_dnn_model(input_dim, hidden_layers=[256, 128, 64], dropout_rate=0.3):

    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.BatchNormalization()
    ])

    for i, units in enumerate(hidden_layers):
        model.add(layers.Dense(units, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))


    model.add(layers.Dense(1, activation='sigmoid'))

    return model


dnn_model = create_dnn_model(X_train.shape[1])

dnn_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=BinaryCrossentropy(),
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

print("Model Structure:")
dnn_model.summary()

print("Training")
dnn_callbacks = [
    callbacks.EarlyStopping(patience=20, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(patience=10, factor=0.5),
    callbacks.ModelCheckpoint('best_dnn_model.h5', save_best_only=True)
]

dnn_history = dnn_model.fit(
    X_train, y_train,
    validation_data=(X_vol, y_vol),
    epochs=50,
    batch_size=32,
    callbacks=dnn_callbacks,
    verbose=1
)


dnn_pred_proba = dnn_model.predict(X_test)
dnn_pred = (dnn_pred_proba > 0.5).astype(int)
dnn_accuracy = accuracy_score(y_test, dnn_pred)
dnn_auc = roc_auc_score(y_test, dnn_pred_proba)

print(f"DNN - Accuracy: {dnn_accuracy:.4f}, AUC: {dnn_auc:.4f}")
print()


Model Structure:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization_4           │ (None, 31)             │           124 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,325 (200.49 KB)

 Trainable params: 50,367 (196.75 KB)

 Non-trainable params: 958 (3.74 KB)

Training
Epoch 1/50
31103/31103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6198 - auc: 0.6576 - loss: 0.6531 - precision: 0.6177 - recall: 0.7443

31103/31103 ━━━━━━━━━━━━━━━━━━━━ 108s 3ms/step - accuracy: 0.6198 - auc: 0.6576 - loss: 0.6531 - precision: 0.6177 - recall: 0.7443 - val_accuracy: 0.6219 - val_auc: 0.6469 - val_loss: 0.6465 - val_precision: 0.6351 - val_recall: 0.8144 - learning_rate: 0.0100
Epoch 2/50
31103/31103 ━━━━━━━━━━━━━━━━━━━━ 98s 3ms/step - accuracy: 0.6378 - auc: 0.6792 - loss: 0.6399 - precision: 0.6312 - recall: 0.7652 - val_accuracy: 0.6334 - val_auc: 0.6033 - val_loss: 0.6698 - val_precision: 0.6310 - val_recall: 0.8823 - learning_rate: 0.0100
Epoch 3/50
31103/31103 ━━━━━━━━━━━━━━━━━━━━ 98s 3ms/step - accuracy: 0.6393 - auc: 0.6812 - loss: 0.6392 - precision: 0.6334 - recall: 0.7615 - val_accuracy: 0.5752 - val_auc: 0.5450 - val_loss: 1.0652 - val_precision: 0.6056 - val_recall: 0.7623 - learning_rate: 0.0100
Epoch 4/50
31103/31103 ━━━━━━━━━━━━━━━━━━━━ 97s 3ms/step - accuracy: 0.6399 - auc: 0.6831 - loss: 0.6378 - precision: 0.6337 - recall: 0.7622 - val_accuracy: 0.6099 - val_auc: 0.5757 - val_loss: 0.